# Custom `%%sql` cell magic

Use the `%%sql` cell magic to streamline [Spark SQL](https://spark.apache.org/docs/latest/sql-ref.html) interactions with structured data.

Here is an example that details how to use PySpark DataFrames to read, write, transform, and analyze data using SQL.

## Create a sample PySpark DataFrame

In [1]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import StructType


DATA_SCHEMA = {
    "type": "struct",
    "fields": [
        {"name": "dummy_col01", "type": "integer", "nullable": True, "metadata": {}},
        {"name": "dummy_col02", "type": "string", "nullable": True, "metadata": {}},
    ],
}
DATA_STRUCT = StructType().fromJson(DATA_SCHEMA)

RAW_DATA = [
    (1, "dummy_col02_val0000000001"),
    (1, "dummy_col02_val0000000002"),
    (2, "dummy_col02_val0000000002"),
    (2, "dummy_col02_val0000000003"),
]

df: DataFrame = spark.createDataFrame(data=RAW_DATA, schema=DATA_STRUCT)

## Create a PySpark DataFrame [temporary view](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.createOrReplaceTempView.html#pyspark.sql.DataFrame.createOrReplaceTempView)

In [2]:
df.createOrReplaceTempView("dummy")

In [3]:
%%sql

SHOW TABLES

24/03/31 02:08:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/03/31 02:08:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/03/31 02:08:49 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/03/31 02:08:49 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@127.0.0.1
24/03/31 02:08:49 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/03/31 02:08:49 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


namespace,tableName,isTemporary
,dummy,false


## `%%sql` cell magic interactions

### [SQL data retrieval](https://spark.apache.org/docs/latest/sql-ref-syntax.html#data-retrieval-statements)

`SELECT` statements can be used directly against the view:

In [4]:
%%sql

SELECT * FROM dummy

dummy_col01,dummy_col02
1,dummy_col02_val00...
1,dummy_col02_val00...
2,dummy_col02_val00...
2,dummy_col02_val00...


Prettier, HTML-based output can be used with the `%%sql_display` magic:

In [5]:
%%sql_display

SELECT * FROM dummy

,dummy_col01,dummy_col02
0,1,dummy_col02_val0000000001
1,1,dummy_col02_val0000000002
2,2,dummy_col02_val0000000002
3,2,dummy_col02_val0000000003


In [6]:
rows = %sql SELECT * FROM dummy

rows.toPandas() if rows.count() > 0 else display("Table is empty")

,dummy_col01,dummy_col02
0,1,dummy_col02_val0000000001
1,1,dummy_col02_val0000000002
2,2,dummy_col02_val0000000002
3,2,dummy_col02_val0000000003


### [DML statements](https://spark.apache.org/docs/latest/sql-ref-syntax.html#dml-statements)

Use DML statements to add, change or delete data.

First, create a table from the existing view. Here we are using PySpark with [Delta Lake](https://docs.delta.io/latest/quick-start.html#set-up-apache-spark-with-delta-lake):

In [7]:
%%sql

DROP TABLE IF EXISTS tester

""


In [8]:
%%sql

CREATE TABLE tester USING delta AS SELECT * FROM dummy

24/03/31 02:08:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/03/31 02:09:02 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`tester` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/03/31 02:09:02 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/03/31 02:09:02 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/03/31 02:09:02 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/03/31 02:09:02 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


""


In [9]:
%%sql_display

SHOW TABLES

,namespace,tableName,isTemporary
0,default,tester,False
1,,dummy,False


#### Insert

Add a new row to the `tester` table:

In [10]:
%%sql

INSERT INTO tester VALUES
    (3, "dummy_col02_val0000000003")

""


In [11]:
%%sql_display

SELECT * FROM tester ORDER BY dummy_col01

,dummy_col01,dummy_col02
0,1,dummy_col02_val0000000001
1,1,dummy_col02_val0000000002
2,2,dummy_col02_val0000000003
3,2,dummy_col02_val0000000002
4,3,dummy_col02_val0000000003


#### Update

Update existing row in the `tester` table:

In [12]:
%%sql

UPDATE tester SET dummy_col02 = 'dummy_col02_val0000000004' WHERE dummy_col01 = 3

num_affected_rows
1


In [13]:
%%sql_display

SELECT * FROM tester ORDER BY dummy_col01

,dummy_col01,dummy_col02
0,1,dummy_col02_val0000000001
1,1,dummy_col02_val0000000002
2,2,dummy_col02_val0000000003
3,2,dummy_col02_val0000000002
4,3,dummy_col02_val0000000004


#### Delete

Delete the new row:

In [14]:
%%sql

DELETE FROM tester WHERE dummy_col01 = 3

num_affected_rows
1


In [15]:
%%sql_display

SELECT * FROM tester ORDER BY dummy_col01

,dummy_col01,dummy_col02
0,1,dummy_col02_val0000000001
1,1,dummy_col02_val0000000002
2,2,dummy_col02_val0000000003
3,2,dummy_col02_val0000000002


Delete row with a wildcard match:

In [16]:
%%sql

DELETE FROM tester WHERE dummy_col02 LIKE '%val0000000001'

num_affected_rows
1


In [17]:
%%sql_display

SELECT * FROM tester ORDER BY dummy_col01

,dummy_col01,dummy_col02
0,1,dummy_col02_val0000000002
1,2,dummy_col02_val0000000003
2,2,dummy_col02_val0000000002


## `%sql` line magic interactions

Returns a PySpark DataFrame for lazy evaluation of the SQL that can be used in other Python expressions. For example:

In [18]:
from pyspark.sql import DataFrame

df: DataFrame = %sql SELECT * FROM tester
df.show()

+-----------+--------------------+
|dummy_col01|         dummy_col02|
+-----------+--------------------+
|          2|dummy_col02_val00...|
|          1|dummy_col02_val00...|
|          2|dummy_col02_val00...|
+-----------+--------------------+

